
### Entrada
- x1: Local da Coleta em km
- x2: Nivel da Mare em m
- x3: Umidade do Ar (%)
- x4: Índice Pluvimétrico (mm)¹
- x5: Temperatura (°C)

### Saida
- y1: pH
- y2: Condutividade Elétrica (mS.cmˉ¹)
- y3: Oxigênio Dissolvido (mg.Lˉ¹)
- y4: Sólidos Totais Dissolvidos (ppm)


In [2]:
 !pip install --upgrade pip
!pip install tensorflow 
!pip install openpyxl

In [3]:
import os

os.chdir("./content")

lm_dir = "tf-levenberg-marquardt"
if not os.path.exists(lm_dir):
  !git clone https://github.com/fabiodimarco/$lm_dir

os.chdir(lm_dir)

In [4]:
import numpy as np
import pandas as pd

def create_dataframe(output, label):
    df = pd.read_excel("../../../data.xlsx")
    x1, x2, x3, x4, x5  = df['x1'], df['x2'], df['x3'], df['x4'], df['x5']

    input = np.vstack([x1, x2, x3, x4, x5 ]).T
    output = np.array(df[output])
    return output, input

output, input = create_dataframe(output="y4", label="solidos")

/tmp/ipykernel_204524/178310800.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [5]:
import tensorflow as tf
import numpy as np
from keras import regularizers
from keras import initializers
import levenberg_marquardt as lm

# layers, neurons
class ShuffleArchitecture:
    def __init__(self, input_size, hidden_sizes, output_size, act_h, act_o, param_reg):
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size
        self.act_h = act_h
        self.act_o = act_o
        self.regularizer = regularizers.L2(param_reg)
        self.initializer = initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=np.random.randint(1, 10000))

    def set_architecture(self):
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.Dense(self.hidden_sizes[0],
                        input_shape=(self.input_size,),
                        activation=self.act_h,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,                        
                        ))  # input layer
        
        for size in self.hidden_sizes[1:]:  # hidden layers
            self.model.add(tf.keras.layers.Dense(size,
                            activation=self.act_h,
                            kernel_regularizer=self.regularizer,
                            kernel_initializer=self.initializer,  
                        ))

        self.model.add(tf.keras.layers.Dense(self.output_size,
                        activation=self.act_o,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,  
                        ))  # output layer

    def create_model(self, _learning_rate):
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=_learning_rate),
            loss=tf.keras.losses.MeanSquaredError())

        self.lm_model = lm.ModelWrapper(
            tf.keras.models.clone_model(self.model))

        self.lm_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=_learning_rate),
            loss=lm.MeanSquaredError())
        return(self.lm_model)

2024-03-26 03:00:57.196071: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-26 03:00:57.255380: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-26 03:00:57.255426: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-26 03:00:57.256972: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-26 03:00:57.266186: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-26 03:00:57.268403: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [6]:
import tensorflow as tf
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score, mean_squared_error


class TrainWithSmallDataset:
    def __init__(self, batch_size=1000):
        self.batch_size = batch_size
        self.betters = []
    
    def create_dataset(self, input, output):
      input = tf.expand_dims(tf.cast(input, tf.float32), axis=-1)
      output = tf.expand_dims(tf.cast(output, tf.float32), axis=-1)

      dataset = tf.data.Dataset.from_tensor_slices((input, output))
      dataset = dataset.shuffle(len(input))
      dataset = dataset.batch(self.batch_size).cache()
      dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
      return (dataset, input, output)

    def split_dataset(self, input, output):
      input_train, input_vt, output_train, output_vt = train_test_split(input, output, test_size=0.3, shuffle = True)
      input_val, input_test, output_val, output_test = train_test_split(input_vt, output_vt, test_size=0.5, shuffle = True)

      self.train_dataset, self.train_input, self.train_output = self.create_dataset(input_train, output_train)
      self.val_dataset, self.val_input, self.val_output = self.create_dataset(input_val, output_val)
      self.test_dataset, self.test_input, self.test_output = self.create_dataset(input_test, output_test)
      self.vt_dataset, self.vt_input, self.vt_output = self.create_dataset(input_vt, output_vt)
      self.dataset, self.input, self.output = self.create_dataset(input, output)

      self._train = (input_train, output_train)
      self._vt = (input_vt, output_vt)
      self._val = (input_val, output_val)
      self._test = (input_test, output_test)
 
 
    
    def train_using_lm(self, train_dataset, epochs=1000):
      early_stopping_monitor = EarlyStopping(monitor='val_loss',
                                              patience=6,
                                              restore_best_weights=True)

      t2_start = time.perf_counter()
      self.results = self.lm_model.fit(train_dataset,
                                            epochs=epochs,
                                            validation_data=self.val_dataset,
                                            callbacks=[early_stopping_monitor],
                                            verbose=0)
      t2_stop = time.perf_counter()
      print("Elapsed time: ", t2_stop - t2_start)
      print ("Stopped at epoch: ", early_stopping_monitor.stopped_epoch)

    def get_metrics(self):
          self.test_prediction = self.lm_model.predict(self.test_input)

          pred = self.lm_model.predict(self.input).flatten()
          test_pred = self.test_prediction.flatten()
          val_pred = self.lm_model.predict(self.val_input).flatten()
          vt_pred = self.lm_model.predict(self.vt_input).flatten()

          r2 = r2_score(self.output, pred)
          r2_test = r2_score(self.test_output, test_pred)
          r2_val = r2_score(self.val_output, val_pred)
          r2_vt = r2_score(self.vt_output, vt_pred)

          mse = mean_squared_error(self.output, pred)
          mse_test = mean_squared_error(self.test_output, test_pred)
          mse_val = mean_squared_error(self.val_output, val_pred)
          mse_vt = mean_squared_error(self.vt_output, vt_pred)

          metrics = {
                          'r2': r2,
                          'r2_test': r2_test,
                          'r2_val': r2_val,
                          'r2_vt': r2_vt,
                          'mse': mse,
                          'mse_test': mse_test,
                          'mse_val': mse_val,
                          'mse_vt': mse_vt
                          }

          return metrics

In [7]:
from itertools import product
import pickle

class Tester:
  def __init__(self, input, output, run_times=500, dataset_run_times=10):
    self.run_times = run_times
    self.better_metrics = {}
    self.dataset_run_times = dataset_run_times
    self.input, self.output = input, output
  
  def setArchitecure(self, trainer, _hidden_sizes, _pg, _lr):
    shuffler = ShuffleArchitecture(input_size=5,
                                    hidden_sizes=_hidden_sizes,
                                    output_size=1,
                                    act_h='tanh',
                                    act_o='linear',
                                    param_reg=_pg)
    shuffler.set_architecture()    
    trainer.lm_model = shuffler.create_model(_lr)

  def Train(self, trainer, epochs=1000):
    trainer.train_using_lm(trainer.train_dataset, epochs=epochs)
    return(trainer.get_metrics(), trainer.lm_model)

  def SaveModelWeights(self, model, fileName):
    path = f"../models/{fileName}.keras"
    open(path,'w').close()
    model.save_weights(path) 

  def SaveDataset(self, trainer, fileName):
    path = f"../dataset/{fileName}.pkl" 
    with open(path, 'wb') as f:
      pickle.dump((trainer._train, trainer._vt, trainer._val, trainer._test), f)


  def LoopWeights(self, sort_by, boundarie, trainer, idx):
    better_model = 0
    save = False

    for i in range(self.run_times):
      print (f"+++++++++++ [{idx}] | {i + 1} ++++++++++++++++++")
      metrics, model = self.Train(trainer)
      if (metrics[sort_by] >= boundarie): # should be <= to descending metrics
        fileName = f"model_{idx}_{better_model}"
        self.SaveModelWeights(model, fileName)
        self.better_metrics[fileName] = metrics
        better_model += 1
        save = True
    
    return(save)

# Init
  def Loop(self, sort_by, boundarie, hidden_sizes, regularizers, learning_rate):
    trainer = TrainWithSmallDataset()

    for count, (hidden_size, reg, lr) in enumerate(product(hidden_sizes, regularizers, learning_rate), start=1):
      header =  f"Hidden Size={hidden_size}, regularizer={reg}, learning_rate={lr}"
      print(f"Testando combinacao{count}: {header}")
      self.setArchitecure(trainer, hidden_size, reg, lr)
      for j in range(self.dataset_run_times):
        trainer.split_dataset(self.input, self.output)
        if (self.LoopWeights(sort_by, boundarie, trainer, f"{count}_{j}") == True):
          self.SaveDataset(trainer, f"dataset_{count}_{j}")
          self.DisplayBetterResults(sort_by, header, f"{count}_{j}")
        self.better_metrics = {}

  def DisplayBetterResults(self, sort_by, header, dataset=0):
    df = pd.DataFrame.from_dict(self.better_metrics, orient='index')
    df = df.sort_values([sort_by])
    display(df)
    path = f'../results/metrics_{dataset}'
    df.to_excel(f"{path}.xlsx", index=True)
    print(f"DataFrame salvo em {path}")
    with open(f"{path}.txt", 'w') as arquivo:
      arquivo.write(header)

In [8]:
tester = Tester(input, output, run_times=25, dataset_run_times=10)

In [9]:
tester.Loop(sort_by='r2',
            boundarie=0.5,
            hidden_sizes = [[15], [20] ,[25], [30], [35], [40], [45]],
            regularizers=[0.02, 0.1],
            learning_rate=[0.1,0.01])

Testando combinacao1: Hidden Size=[15], regularizer=0.02, learning_rate=0.1
+++++++++++ [1_0] | 1 ++++++++++++++++++


Elapsed time:  1.225049468999714
Stopped at epoch:  8
1/1 [==============================] - 0s 21ms/step
+++++++++++ [1_0] | 2 ++++++++++++++++++
Elapsed time:  0.3290063819986244
Stopped at epoch:  6
1/1 [==============================] - 0s 21ms/step
+++++++++++ [1_0] | 3 ++++++++++++++++++
Elapsed time:  0.9783058550001442
Stopped at epoch:  17
1/1 [==============================] - 0s 18ms/step
+++++++++++ [1_0] | 4 ++++++++++++++++++
Elapsed time:  0.38920221500120533
Stopped at epoch:  6
1/1 [==============================] - 0s 25ms/step
+++++++++++ [1_0] | 5 ++++++++++++++++++
Elapsed time:  0.3522030010008166
Stopped at epoch:  6
1/1 [==============================] - 0s 18ms/step
+++++++++++ [1_0] | 6 ++++++++++++++++++
Elapsed time:  0.45254342299995187
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 7 ++++++++++++++++++
Elapsed time:  0.3083490270000766
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_1_0_0,0.504918,0.481842,0.198182,0.367348,59.707008,37.129227,74.697235,54.808292


DataFrame salvo em ../results/metrics_1_0
+++++++++++ [1_1] | 1 ++++++++++++++++++
Elapsed time:  0.28758822200143186
Stopped at epoch:  6
1/1 [==============================] - 0s 17ms/step
+++++++++++ [1_1] | 2 ++++++++++++++++++
Elapsed time:  0.23259731300095154
Stopped at epoch:  6
1/1 [==============================] - 0s 22ms/step
+++++++++++ [1_1] | 3 ++++++++++++++++++
Elapsed time:  0.6125069429999712
Stopped at epoch:  9
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_1] | 4 ++++++++++++++++++
Elapsed time:  0.38742009099951247
Stopped at epoch:  6
1/1 [==============================] - 0s 23ms/step
+++++++++++ [1_1] | 5 ++++++++++++++++++
Elapsed time:  1.0245247000002564
Stopped at epoch:  18
1/1 [==============================] - 0s 17ms/step
+++++++++++ [1_1] | 6 ++++++++++++++++++
Elapsed time:  0.3401999210000213
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_1] | 7 ++++++++++++++++++
Elapsed time:  0.34404320

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_1_2_1,0.501463,0.662562,0.414489,0.514106,60.123726,23.107376,74.451912,47.269562
model_1_2_0,0.503048,0.710330,0.428730,0.540667,59.932529,19.836330,72.641052,44.685616


...

DataFrame salvo em ../results/metrics_28_7
+++++++++++ [28_8] | 1 ++++++++++++++++++
Elapsed time:  0.22233902900006797
Stopped at epoch:  7
1/1 [==============================] - 0s 18ms/step
+++++++++++ [28_8] | 2 ++++++++++++++++++
Elapsed time:  0.23839550000047893
Stopped at epoch:  6
1/1 [==============================] - 0s 20ms/step
+++++++++++ [28_8] | 3 ++++++++++++++++++
Elapsed time:  0.463828554000429
Stopped at epoch:  13
1/1 [==============================] - 0s 17ms/step
+++++++++++ [28_8] | 4 ++++++++++++++++++
Elapsed time:  0.2131077349986299
Stopped at epoch:  6
1/1 [==============================] - 0s 17ms/step
+++++++++++ [28_8] | 5 ++++++++++++++++++
Elapsed time:  0.24910835000082443
Stopped at epoch:  7
1/1 [==============================] - 0s 17ms/step
+++++++++++ [28_8] | 6 ++++++++++++++++++
Elapsed time:  0.21172818200102483
Stopped at epoch:  6
1/1 [==============================] - 0s 17ms/step
+++++++++++ [28_8] | 7 ++++++++++++++++++
Elapsed time:  0.

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_28_8_4,0.500460,-1.003110,0.496935,-0.113066,60.244682,142.256561,56.846245,102.063354
model_28_8_10,0.500539,-1.248830,0.412796,-0.262613,60.235188,159.707092,66.353882,115.776169
model_28_8_5,0.501396,-1.114713,0.454024,-0.183713,60.131771,150.182404,61.695175,108.541344
model_28_8_9,0.501498,-1.241161,0.427278,-0.251070,60.119499,159.162460,64.717400,114.717712
model_28_8_8,0.502331,-1.238472,0.432126,-0.247157,60.019009,158.971497,64.169594,114.358849
model_28_8_2,0.502693,-0.765992,0.614013,0.052053,59.975342,125.416954,43.616364,86.922668
model_28_8_7,0.505253,-1.157875,0.441314,-0.208781,59.666607,153.247620,63.131348,110.839935
model_28_8_6,0.505353,-1.134639,0.451961,-0.193079,59.654568,151.597488,61.928223,109.400208
model_28_8_3,0.522654,-0.580918,0.607334,0.124066,57.568123,112.273384,44.371086,80.319366
model_28_8_1,0.523458,-0.413966,0.619619,0.199645,57.471157,100.416809,42.982979,73.389099


DataFrame salvo em ../results/metrics_28_8
+++++++++++ [28_9] | 1 ++++++++++++++++++
Elapsed time:  0.5628406820014789
Stopped at epoch:  18
1/1 [==============================] - 0s 16ms/step
+++++++++++ [28_9] | 2 ++++++++++++++++++
Elapsed time:  0.21857231500143826
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [28_9] | 3 ++++++++++++++++++
Elapsed time:  1.2676100200005749
Stopped at epoch:  40
1/1 [==============================] - 0s 17ms/step
+++++++++++ [28_9] | 4 ++++++++++++++++++
Elapsed time:  0.7119203109996306
Stopped at epoch:  21
1/1 [==============================] - 0s 23ms/step
+++++++++++ [28_9] | 5 ++++++++++++++++++
Elapsed time:  1.0545260919989232
Stopped at epoch:  32
1/1 [==============================] - 0s 17ms/step
+++++++++++ [28_9] | 6 ++++++++++++++++++
Elapsed time:  1.5623379230000864
Stopped at epoch:  46
1/1 [==============================] - 0s 16ms/step
+++++++++++ [28_9] | 7 ++++++++++++++++++
Elapsed time:  

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_28_9_0,0.500610,0.779421,0.219204,0.568725,60.226624,13.471476,64.078720,37.286526
model_28_9_19,0.510284,0.139134,-0.937057,-0.187231,59.059902,52.575993,158.971222,102.643822
model_28_9_18,0.511753,0.136520,-0.917430,-0.179454,58.882710,52.735630,157.360489,101.971397
model_28_9_1,0.513536,0.391420,-0.082066,0.289079,58.667664,37.168064,88.803459,61.463696
model_28_9_17,0.517078,0.141269,-0.852497,-0.148726,58.240547,52.445633,152.031540,99.314766
model_28_9_16,0.520549,0.116427,-0.782580,-0.126739,57.821918,53.962776,146.293533,97.413902
model_28_9_14,0.522691,0.075771,-0.703134,-0.106391,57.563629,56.445808,139.773544,95.654648
model_28_9_15,0.524119,0.112015,-0.739063,-0.108917,57.391384,54.232224,142.722137,95.873009
model_28_9_7,0.534965,0.087191,-0.210269,0.117991,56.083408,55.748348,99.324860,76.255402
model_28_9_6,0.537338,0.064669,-0.093617,0.161706,55.797199,57.123798,89.751465,72.475952


DataFrame salvo em ../results/metrics_28_9
